In [47]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

from process import preprocess_all_files

data = preprocess_all_files()


# Extract X and Y from the data
X_data = [item[0] for item in data]
Y_data = [item[1] for item in data]

# Pad or truncate sequences to a fixed length
# max_sequence_length = 999  # Maximum sequence length
# X_data = pad_sequences(X_data, maxlen=max_sequence_length, padding='post', dtype='float32')

# Convert lists to numpy arrays
X_data = np.array(X_data)
Y_data = np.array(Y_data)

# Split data
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

# Define model
model = Sequential()
# model.add(Masking(mask_value=0.0, input_shape=(max_sequence_length, 7)))
model.add(LSTM(units=50))
model.add(Dense(units=10, activation='linear'))  

# model = Sequential([
#     LSTM(64, input_shape=(3827, 999, 7), return_sequences=True),
#     Dropout(0.2),
#     LSTM(64, return_sequences=True),
#     Dropout(0.2),
#     Dense(5, activation='sigmoid')
# ])

# Compile modle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2) 

# Evaluate model
loss = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")

<class 'numpy.float64'>
